In [3]:
import os
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser


In [4]:

# --- CONFIGURACIÓN ---
MODEL_NAME = "llama3.2"  
VECTOR_DB_PATH = "./chroma_db"


In [5]:

# --- PASO 1: CARGAR DOCUMENTOS ---
def cargar_documentos():
    print(" Cargando documentos...")
    docs = []
    
    # Cargar Filosofía
    if os.path.exists("data/teoria/filosofia_core.txt"):
        loader_teoria = TextLoader("data/teoria/filosofia_core.txt", encoding="utf-8")
        docs_teoria = loader_teoria.load()
        # Añadimos metadatos para saber que esto es teoría
        for doc in docs_teoria:
            doc.metadata["source_type"] = "teoria_filosofica"
        docs.extend(docs_teoria)
    else:
        print(" ALERTA: No se encontró filosofia_core.txt")

    # Cargar Comentarios Gen Z
    if os.path.exists("data/redes_sociales/comentarios_genz.txt"):
        loader_social = TextLoader("data/redes_sociales/comentarios_genz.txt", encoding="utf-8")
        docs_social = loader_social.load()
        # Añadimos metadatos para saber que esto es evidencia empírica
        for doc in docs_social:
            doc.metadata["source_type"] = "redes_sociales"
        docs.extend(docs_social)
    else:
        print(" ALERTA: No se encontró comentarios_genz.txt")
    
    return docs



In [6]:
# --- PASO 2: CREAR INDEX (EMBEDDINGS) ---
def preparar_rag():
    docs = cargar_documentos()
    
    # Dividir texto en fragmentos (chunks)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    splits = text_splitter.split_documents(docs)
    print(f" Documentos divididos en {len(splits)} fragmentos.")

    # Crear Embeddings (usamos un modelo ligero y rápido de HuggingFace local)
    print(" Generando base de datos vectorial (esto puede tardar unos segundos)...")
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    
    # Crear Vector Store (Persistente en disco para no rehacerlo siempre)
    vectorstore = Chroma.from_documents(
        documents=splits, 
        embedding=embeddings, 
        persist_directory=VECTOR_DB_PATH
    )
    print(" Base de datos vectorial lista y guardada.")
    return vectorstore



In [7]:
# --- PASO 3: CONFIGURAR EL CEREBRO (LLAMA 3.2) ---
def consultar_rag(pregunta, vectorstore):
    # Definir el modelo
    llm = ChatOllama(model=MODEL_NAME, temperature=0.3) # Temp baja para ser analítico
    
    # Definir el recuperador (Retriever)
    retriever = vectorstore.as_retriever(search_kwargs={"k": 5}) # Traer los 5 fragmentos más relevantes

    # Prompt del sistema (Instrucciones para Llama)
    template = """Eres un filósofo de datos experto en Byung-Chul Han, Foucault y Bauman.
    Tu tarea es responder a la pregunta basándote SOLAMENTE en el contexto proporcionado.
    
    Instrucciones:
    1. Analiza los comentarios de redes sociales en el contexto.
    2. Únelos con los conceptos filosóficos de la teoría (panóptico, sociedad del cansancio, modernidad líquida).
    3. Si no encuentras la respuesta en el contexto, di "No tengo suficientes datos para responder esto".
    4. Cita ejemplos concretos de los comentarios si los hay.

    Contexto: {context}

    Pregunta: {question}
    """
    prompt = ChatPromptTemplate.from_template(template)

    # Crear la cadena (Chain)
    rag_chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

    print(f"\n🤔 Analizando: '{pregunta}'...")
    print("-" * 50)
    resultado = rag_chain.invoke(pregunta)
    return resultado



In [9]:
# 1. Preparar o cargar la BD
if os.path.exists(VECTOR_DB_PATH):
    print("📂 Cargando base de datos existente...")
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vectorstore = Chroma(persist_directory=VECTOR_DB_PATH, embedding_function=embeddings)
else:
    vectorstore = preparar_rag()

# 2. Prueba rápida
pregunta_prueba = "¿Qué expresiones o términos utiliza la Gen Z para describir el vacío existencial en redes sociales?"
respuesta = consultar_rag(pregunta_prueba, vectorstore)

print("\n📝 RESPUESTA DE LLAMA 3.2:")
print(respuesta)

📂 Cargando base de datos existente...

🤔 Analizando: '¿Qué expresiones o términos utiliza la Gen Z para describir el vacío existencial en redes sociales?'...
--------------------------------------------------

📝 RESPUESTA DE LLAMA 3.2:
Basándome en los comentarios de redes sociales proporcionados, puedo identificar algunas expresiones y términos utilizados por la Gen Z para describir el vacío existencial en redes sociales. A continuación, te presento algunos ejemplos:

1. "Falta de dignidad" - En el comentario número 1, se menciona que las familias hacen ciudadanos débiles y carentes de dignidad para luchar por su libertad. Esto sugiere que la Gen Z percibe a sí misma como débil y sin dignidad en comparación con sus antepasados.
2. "Situaciones difíciles y peligrosas" - En el mismo comentario, se menciona que los que pelean por un mundo libre de sistemas putrefactos son perseguidos y se verán envueltos en situaciones difíciles y peligrosas. Esto sugiere que la Gen Z percibe a sí misma 